In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
train_df = pd.read_csv('/kaggle/input/forest-type-classification-spai/train.csv')
test_df = pd.read_csv('/kaggle/input/forest-type-classification-spai/test.csv')
def calculate_vegetation_indices2(df):
    df['RVI'] = df['b8'] / df['b4']
    df['PSSRa'] = df['b7'] / df['b4']
    df['GOSSAN'] = df['b11'] / df['b4']

    SWIRmin = 0.378
    SWIRmax = 0.397
    SWIIRmin = 0.027
    df['NDVI'] = (df['b8'] - df['b4']) / (df['b8'] + df['b4'])
    df['NDVI_2'] = (df['b8'] - df['b5']) / (df['b8'] + df['b5'])
    df['NDVIc'] = (df['b8'] - df['b4']) / (df['b8'] + df['b4']) * (1.0 - (df['b12'] - SWIIRmin) / (SWIRmax - SWIRmin))

    df['TNDVI'] = np.sqrt(df['NDVI'] + 0.5)
    df['NDI45'] = (df['b5'] - df['b4']) / (df['b5'] + df['b4'])
    df['GNDVI'] = (df['b8'] - df['b3']) / (df['b8'] + df['b3']) # Green Normalized Difference Vegetation Index: General formula: (NIR - [540:570]) / (NIR + [540:570])
    df['MCARI'] = ((df['b5'] - df['b4']) - 0.2 * (df['b5'] - df['b3'])) * (df['b5'] / df['b4'])
    df['LCI'] = (df['b8'] - df['b5']) / (df['b8'] + df['b4']) # Leaf Chlorophyll Index
    df['GBNDVI'] = (df['b8'] - (df['b3'] + df['b2'])) / (df['b8'] + (df['b3']+ df['b2'])) # Green-Blue NDVI
    df['GRNDVI'] = (df['b8'] - (df['b3'] + df['b4'])) / (df['b8'] + (df['b3'] + df['b4'])) # Green-Red NDVI
    df['MVI'] = df['b9'] / df['b11'] # Mid-infrared vegetation index
    df['MGVI'] = -0.386 * df['b3'] - 0.53 * df['b4'] + 0.535 * df['b6'] + 0.532 * df['b9'] # Misra Green Vegetation Index
    df['MYVI'] = 0.723 * df['b3'] - 0.597 * df['b4'] + 0.206 * df['b6'] - 0.278 * df['b9'] # Misra Yellow Vegetation Index
    df['mARI'] = (df['b3']**-1.0 - df['b5']**-1.0) * df['b8'] # Modified anthocyanin reflectance index
    df['BRI'] = ((1.0 / df['b3']) - (1.0 / df['b5'])) / df['b8'] # Browning Reflectance Index
    df['CCCI'] = ((df['b8'] - df['b5']) / (df['b8'] + df['b5'])) / ((df['b8'] - df['b4']) / (df['b8'] + df['b4'])) # Canopy Chlorophyll Content Index
    df['CIgreen'] = df['b8'] / df['b3'] - 1.0 # Chlorophyll Index Green
    df['CIrededge'] = df['b8'] / df['b5'] - 1.0 # Cholorphyll IndexRedEdge
    df['CIred-edge'] = (df['b7'] / df['b5'])**-1.0 # Chlorophyll Red-Edge
    df['CVI'] = df['b8'] * df['b4'] / (df['b3'])**2.0 # Chlorophyll Vegetation Index
    df['CI'] = (df['b4'] - df['b2']) / df['b4'] # Coloration Index
    df['CTVI'] = (((df['b4'] - df['b3']) / (df['b4'] + df['b3'])) + 0.5) / \
                     np.abs(((df['b4'] - df['b3']) / (df['b4'] + df['b3'])) + 0.5) * \
                     np.sqrt(np.abs((((df['b4'] - df['b3']) / (df['b4'] + df['b3']))) + 0.5)) # Corrected Transformed Vegetation Index

    df['CARI'] = (df['b5'] / df['b4']) * np.sqrt(np.power(((df['b5'] - df['b3']) / 150.0 * 670.0 + df['b4'] + (df['b3'] - ((df['b5'] - df['b3']) / 150.0 * 550.0))), 2.0)) / \
                    np.power(((df['b5'] - df['b3']) / np.power(150.0, 2.0) + 1.0), 0.5) # Chlorophyll Absorption Ratio Index
    
    a = 0.496
    df['CARI_2'] = (np.abs(((df['b5'] - df['b3']) / 150.0 * df['b4'] + df['b4'] + df['b3'] - (a * df['b3']))) / \
                    np.sqrt(np.power(a, 2.0) + 1.0)) * (df['b5'] / df['b4']) # Chlorophyll Absorption Ratio Index 2
    df['Chlgreen'] = (df['b7'] / df['b3'])**-1.0

    df['MCARI'] = ((df['b5'] - df['b4']) - 0.2 * (df['b5'] - df['b3'])) * (df['b5'] / df['b4']) # Modified Chlorophyll Absorption in Reflectance Index
    df['MCARI_1'] = 1.2 * (2.5 * (df['b8'] - df['b4']) - 1.3 * (df['b8'] - df['b3'])) # Modified Chlorophyll Absorption in Reflectance Index 1
    df['MCARI_2'] = 1.5 * (2.5 * (df['b8'] - df['b4']) - 1.3 * (df['b8'] - df['b3'])) / \
                     np.sqrt(np.power((2.0 * df['b8'] + 1.0), 2.0) - (6.0 * df['b8'] - 5.0 * np.sqrt(df['b4'])) - 0.5) # Modified Chlorophyll Absorption in Reflectance Index 2
    df['mNDVI'] = (df['b8'] - df['b4']) / (df['b8'] + df['b4'] - 2.0 * df['b1']) # Modified Normalized Difference Vegetation Index

    L = 0
    df['SAVI'] = (1 + L) * (df['b8'] - df['b4']) / (df['b8'] + df['b4'] + L)
    df['GSAVI'] = (df['b8'] - df['b3']) / (df['b8'] + df['b3'] + L) * (1.0 + L)
    df['MSAVI'] = (2 * df['b8'] + 1 - np.sqrt((2 * df['b8'] + 1)**2 - 8 * (df['b8'] - df['b4']))) / 2
    df['OSAVI'] = (1.0 + 0.16) * (df['b8'] - df['b4']) / (df['b8'] + df['b4'] + 0.16)

    df['EVI'] = 2.5 * (df['b8'] - df['b4']) / (df['b8'] + 6 * df['b4'] - 7.5 * df['b2'] + 1) # General formula: 2.5 * (NIR - RED) / ((NIR + 6*RED - 7.5*BLUE) + 1)
    df['EVI_2'] = 2.4 * (df['b8'] - df['b4']) / (df['b8'] + df['b4'] + 1) # General formula: 2.4 * (NIR - RED) / (NIR + RED + 1)
    df['EVI_2_2'] = 2.5 * (df['b8'] - df['b4']) / (df['b8'] + 2.4 * df['b4'] + 1) #General formula: 2.5 * (NIR - RED) / (NIR + 2.4 * RED + 1)
    df['EVI_8A'] = 2.5 * (df['b8_a'] - df['b4']) / (df['b8_a'] + 6 * df['b4'] - 7.5 * df['b2'] + 1) # General formula: 2.5 * (NIR - RED) / ((NIR + 6*RED - 7.5*BLUE) + 1)
    df['EVI_8A_2'] = 2.4 * (df['b8_a'] - df['b4']) / (df['b8_a'] + df['b4'] + 1) # General formula: 2.4 * (NIR - RED) / (NIR + RED + 1)
    df['EVI_8A_2_2'] = 2.5 * (df['b8_a'] - df['b4']) / (df['b8_a'] + 2.4 * df['b4'] + 1) #General formula: 2.5 * (NIR - RED) / (NIR + 2.4 * RED + 1)
    df['GLI'] = (2.0 * df['b3'] - df['b4'] - df['b2']) / (2.0 * df['b3'] + df['b4'] + df['b2']) # Green Leaf Index: General formula: (2 * GREEN - RED - BLUE)/(2 * GREEN + RED + BLUE)
    df['GARI'] = (df['b8'] - (df['b3'] - (df['b2'] - df['b4']))) / (df['b8'] - (df['b3'] + (df['b2'] - df['b4']))) # Green atmospherically resistant vegetation index: General formula: ( NIR - (GREEN-(BLUE- RED))) /  ( NIR- (GREEN+(BLUE - RED)))

    ar = 0.374
    b = 0.735
    df['PVI'] = (1.0 / np.sqrt(np.power(0.149, 2.0) + 1.0)) * (df['b8'] - ar - b)
    df['GCI'] = df['b8'] / df['b3'] - 1
    df['PRI'] = (df['b3'] - df['b5']) / (df['b3'] + df['b5'])
    df['MSI'] = df['b11'] / df['b8']
    df['NBR'] = (df['b8'] - df['b12']) / (df['b8'] + df['b12'])
    df['ARI'] = (1 / df['b3']) - (1 / df['b5'])
    df['RDVI'] =  (df['b8'] - df['b4']) / np.power((df['b8'] + df['b4']), 0.5)
    df['NLI'] = (np.power(df['b8'], 2.0) - df['b4']) / (np.power(df['b8'], 2.0) + df['b4']) # Nonlinear Vegetation Index
    
    df['DVI'] = df['b8'] - df['b4']
    df['S2REP'] = 705 + 35 * ((df['b4'] + df['b7']) / 2 - df['b5']) / (df['b6'] - df['b5'])
    df['IRECI'] = (df['b7'] - df['b4']) / (df['b5'] / df['b6'])
    df['MOIS'] = ((df['b8_a'] - df['b11']) / (df['b8_a'] + df['b11'])) # Moisture Index ideal for finding water stress in plants. It uses the short-wave and near-infrared to generate an index of moisture content. In general, wetter vegetation has higher values. But lower moisture index values suggest plants are under stress from insufficient moisture.
    df['NIR_RATIO_1'] = df['b8'] / df['b4']
    df['NIR_RATIO_2'] =df['b7'] / df['b4']
    df['NDSI'] = (df['b11'] - df['b12']) / (df['b11'] + df['b12']) # Normalized Difference Salinity Index  (abbrv. NDSI)
    # Global Environment Monitoring Index
    df['GEMI'] = ((2.0 * (np.power(df['b8'], 2.0) - np.power(df['b4'], 2.0)) + 1.5 * df['b8'] + 0.5 * df['b4']) / (df['b8'] + df['b4'] + 0.5) * (1.0 - 0.25 * ((2.0 * (np.power(df['b8'], 2.0) - np.power(df['b4'], 2.0)) + 1.5 * df['b8'] + 0.5 * df['b4']) / (df['b8'] + df['b4'] + 0.5))) - ((df['b4'] - 0.125) / (1.0 - df['b4'])))
    df['GEMI_8A'] = ((2.0 * (np.power(df['b8_a'], 2.0) - np.power(df['b4'], 2.0)) + 1.5 * df['b8_a'] + 0.5 * df['b4']) / (df['b8_a'] + df['b4'] + 0.5) * (1.0 - 0.25 * ((2.0 * (np.power(df['b8_a'], 2.0) - np.power(df['b4'], 2.0)) + 1.5 * df['b8_a'] + 0.5 * df['b4']) / (df['b8_a'] + df['b4'] + 0.5))) - ((df['b4'] - 0.125) / (1.0 - df['b4']))) # Self Engineered
    df['GVMI'] = ((df['b8'] + 0.1) - (df['b12'] + 0.02)) / ((df['b8'] + 0.1) + (df['b12'] + 0.02)) # Global Vegetation Moisture Index
    df['GVMI_8A'] = ((df['b8_a'] + 0.1) - (df['b12'] + 0.02)) / ((df['b8_a'] + 0.1) + (df['b12'] + 0.02))

    df['b11/b12'] = df['b11'] / df['b12']
    df['b12/b11'] = df['b12'] / df['b11']
    df['b11-b12'] = df['b11'] - df['b12']
    df['b12-b11'] = df['b12'] - df['b11']
    df['b4-b5'] = df['b4'] - df['b5']
    df['b11-b5'] = df['b11'] - df['b5']
    df['b11/b8'] = df['b11'] / df['b8']
    return df

# Preprocess data
calculate_vegetation_indices2(train_df)
calculate_vegetation_indices2(test_df)

,id,b1,b11,b12,b2,b3,b4,b5,b6,b7,...,GEMI_8A,GVMI,GVMI_8A,b11/b12,b12/b11,b11-b12,b12-b11,b4-b5,b11-b5,b11/b8
0,13467,69,1425,693,312,524,376,847,1821,2356,...,-4.992152e+06,0.548686,0.580512,2.056277,0.486316,732,-732,-471,578,0.599243
1,12719,242,1514,691,343,522,324,718,1730,2178,...,-4.138416e+06,0.563077,0.546890,2.191027,0.456407,823,-823,-394,796,0.612460
2,1054,218,2354,1118,292,596,410,965,2586,3226,...,-1.046070e+07,0.501898,0.530551,2.105546,0.474936,1236,-1236,-555,1389,0.698309
3,13747,350,2013,1134,306,572,475,982,1754,1935,...,-3.291813e+06,0.334714,0.337628,1.775132,0.563338,879,-879,-507,1031,0.884835
4,9453,185,1450,712,293,440,384,673,1487,1965,...,-3.295400e+06,0.513169,0.510995,2.036517,0.491034,738,-738,-289,777,0.655219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,115,447,1686,811,425,661,441,958,2432,2891,...,-7.205528e+06,0.570562,0.588014,2.078915,0.481020,875,-875,-517,728,0.568442
3996,10654,252,2694,1503,470,778,753,1294,2334,2656,...,-6.043458e+06,0.281216,0.362468,1.792415,0.557906,1191,-1191,-541,1400,1.005599
3997,5718,233,1486,618,249,409,260,699,2188,2831,...,-7.982255e+06,0.661184,0.666307,2.404531,0.415882,868,-868,-439,787,0.490429
3998,13054,221,1840,774,245,441,231,703,2491,3453,...,-1.246287e+07,0.618533,0.658730,2.377261,0.420652,1066,-1066,-472,1137,0.560292


In [74]:
columns_to_drop = ['id']
train_df = train_df.drop(columns=columns_to_drop)
test_df = test_df.drop(columns=columns_to_drop)

In [75]:
X = train_df.drop(columns=['nforest_type'])
y = train_df['nforest_type']

In [76]:
# Label encode target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=42)

In [77]:
# Define classifiers
xgb_clf = XGBClassifier()
lgbm_clf = LGBMClassifier()
mlp_clf = MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', random_state=42)

In [78]:
# Create ensemble model
ensemble_clf = VotingClassifier(estimators=[('xgb', xgb_clf), ('lgbm', lgbm_clf), ('mlp', mlp_clf)], voting='hard')

In [79]:
# Create pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Include scaler if needed
    ('ensemble', ensemble_clf)
])

In [80]:
# Train pipeline
pipeline.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022484 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20655
[LightGBM] [Info] Number of data points in the train set: 12922, number of used features: 81
[LightGBM] [Info] Start training from score -1.042267
[LightGBM] [Info] Start training from score -1.622054
[LightGBM] [Info] Start training from score -0.798834


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('scaler', StandardScaler()),
                ('ensemble',
                 VotingClassifier(estimators=[('xgb',
                                               XGBClassifier(base_score=None,
                                                             booster=None,
                                                             callbacks=None,
                                                             colsample_bylevel=None,
                                                             colsample_bynode=None,
                                                             colsample_bytree=None,
                                                             device=None,
                                                             early_stopping_rounds=None,
                                                             enable_categorical=False,
                                                             eval_metric=None,
                                                             feature_types=None,
                                                             gamma=None,
                                                             grow_policy=None,
                                                             impor...
                                                             max_cat_threshold=None,
                                                             max_cat_to_onehot=None,
                                                             max_delta_step=None,
                                                             max_depth=None,
                                                             max_leaves=None,
                                                             min_child_weight=None,
                                                             missing=nan,
                                                             monotone_constraints=None,
                                                             multi_strategy=None,
                                                             n_estimators=None,
                                                             n_jobs=None,
                                                             num_parallel_tree=None,
                                                             random_state=None, ...)),
                                              ('lgbm', LGBMClassifier()),
                                              ('mlp',
                                               MLPClassifier(hidden_layer_sizes=(100,
                                                                                 50),
                                                             random_state=42))]))])

In [81]:
# Evaluate pipeline
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7633587786259542


In [82]:
test_df

,b1,b11,b12,b2,b3,b4,b5,b6,b7,b8,...,GEMI_8A,GVMI,GVMI_8A,b11/b12,b12/b11,b11-b12,b12-b11,b4-b5,b11-b5,b11/b8
0,69,1425,693,312,524,376,847,1821,2356,2378,...,-4.992152e+06,0.548686,0.580512,2.056277,0.486316,732,-732,-471,578,0.599243
1,242,1514,691,343,522,324,718,1730,2178,2472,...,-4.138416e+06,0.563077,0.546890,2.191027,0.456407,823,-823,-394,796,0.612460
2,218,2354,1118,292,596,410,965,2586,3226,3371,...,-1.046070e+07,0.501898,0.530551,2.105546,0.474936,1236,-1236,-555,1389,0.698309
3,350,2013,1134,306,572,475,982,1754,1935,2275,...,-3.291813e+06,0.334714,0.337628,1.775132,0.563338,879,-879,-507,1031,0.884835
4,185,1450,712,293,440,384,673,1487,1965,2213,...,-3.295400e+06,0.513169,0.510995,2.036517,0.491034,738,-738,-289,777,0.655219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,447,1686,811,425,661,441,958,2432,2891,2966,...,-7.205528e+06,0.570562,0.588014,2.078915,0.481020,875,-875,-517,728,0.568442
3996,252,2694,1503,470,778,753,1294,2334,2656,2679,...,-6.043458e+06,0.281216,0.362468,1.792415,0.557906,1191,-1191,-541,1400,1.005599
3997,233,1486,618,249,409,260,699,2188,2831,3030,...,-7.982255e+06,0.661184,0.666307,2.404531,0.415882,868,-868,-439,787,0.490429
3998,221,1840,774,245,441,231,703,2491,3453,3284,...,-1.246287e+07,0.618533,0.658730,2.377261,0.420652,1066,-1066,-472,1137,0.560292


In [83]:
y_pred = pipeline.predict(test_df)
y_pred_labels = label_encoder.inverse_transform(y_pred)
result_df = test_df.copy()
result_df['nforest_type'] = y_pred_labels

In [84]:
result_df.to_csv('NAMOTUSSA2.csv', index=True)

print("Prediction complete. The results have been saved to 'predicted_forest_type.csv'.")

Prediction complete. The results have been saved to 'predicted_forest_type.csv'.


In [85]:
result_df

,b1,b11,b12,b2,b3,b4,b5,b6,b7,b8,...,GVMI,GVMI_8A,b11/b12,b12/b11,b11-b12,b12-b11,b4-b5,b11-b5,b11/b8,nforest_type
0,69,1425,693,312,524,376,847,1821,2356,2378,...,0.548686,0.580512,2.056277,0.486316,732,-732,-471,578,0.599243,DEF
1,242,1514,691,343,522,324,718,1730,2178,2472,...,0.563077,0.546890,2.191027,0.456407,823,-823,-394,796,0.612460,DDF
2,218,2354,1118,292,596,410,965,2586,3226,3371,...,0.501898,0.530551,2.105546,0.474936,1236,-1236,-555,1389,0.698309,MDF
3,350,2013,1134,306,572,475,982,1754,1935,2275,...,0.334714,0.337628,1.775132,0.563338,879,-879,-507,1031,0.884835,DDF
4,185,1450,712,293,440,384,673,1487,1965,2213,...,0.513169,0.510995,2.036517,0.491034,738,-738,-289,777,0.655219,DDF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,447,1686,811,425,661,441,958,2432,2891,2966,...,0.570562,0.588014,2.078915,0.481020,875,-875,-517,728,0.568442,DEF
3996,252,2694,1503,470,778,753,1294,2334,2656,2679,...,0.281216,0.362468,1.792415,0.557906,1191,-1191,-541,1400,1.005599,MDF
3997,233,1486,618,249,409,260,699,2188,2831,3030,...,0.661184,0.666307,2.404531,0.415882,868,-868,-439,787,0.490429,DDF
3998,221,1840,774,245,441,231,703,2491,3453,3284,...,0.618533,0.658730,2.377261,0.420652,1066,-1066,-472,1137,0.560292,MDF
